In [9]:
import json
import numpy as np
from keras.preprocessing.image import load_img, img_to_array
from keras.applications.xception import xception
from keras.utils import to_categorical
from keras.layers import Dense, Flatten
from keras.models import Model
from keras.preprocessing.image import ImageDataGenerator

In [10]:
# os.environ["CUDA_VISIBLE_DEVICES"] = '0'

In [2]:
train_path = "./AgriculturalDisease_trainingset/AgriculturalDisease_train_annotations.json"
validation_path = "./AgriculturalDisease_validationset/AgriculturalDisease_validation_annotations.json"
class_id = "disease_class"
img_id = "image_id"

In [3]:
def extract_info(train_path, validation_path, class_id, img_id, img_num=200):
    with open(train_path) as load_f:
        id_train_dict = json.load(load_f)
        
    with open(validation_path) as load_f:
        id_valid_dict = json.load(load_f)
        

    train_label = []
    train_img_id = []
    valid_label = []
    valid_img_id = []
    
    for train_data in id_train_dict:
        
        train_img_id.append(train_data[img_id])
        train_label.append(train_data[class_id])
        
    for valid_data in id_valid_dict:  
        valid_img_id.append(valid_data[img_id])
        valid_label.append(valid_data[class_id])

        
    return train_label, train_img_id, valid_label, valid_img_id

train_root_path = "./AgriculturalDisease_trainingset/images/"
valid_root_path = "./AgriculturalDisease_validationset/images/"

def load_dataset(train_root_path, valid_root_path):

    
    train_label, train_img_id, valid_label, valid_img_id = extract_info(train_path, validation_path, class_id, img_id)
    train = []
    valid = []
    for train_img in train_img_id:
        train.append(img_to_array(load_img(train_root_path + train_img).resize((128, 128))))
        
    for valid_img in valid_img_id:
        valid.append(img_to_array(load_img(valid_root_path + valid_img).resize((128, 128))))
        
        
    train = np.array(train)
    valid = np.array(valid)

    
    return train, train_label, valid, valid_label

In [4]:
train, train_label, valid, valid_label = load_dataset(train_root_path, valid_root_path)

In [5]:
train.shape, len(train_label)

((32739, 128, 128, 3), 32739)

In [6]:
valid.shape, len(valid_label)

((4982, 128, 128, 3), 4982)

In [7]:
train = xception.preprocess_input(train)
valid = xception.preprocess_input(valid)
train_label = to_categorical(train_label)
valid_label = to_categorical(valid_label)

base_model = xception.Xception(include_top=False, input_shape=(128, 128, 3))
x = base_model.output
x = Flatten()(x)
x = Dense(61, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=x)
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 128, 128, 3)  0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 63, 63, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 63, 63, 32)   128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 63, 63, 32)   0           block1_conv1_bn[0][0]            
__________________________________________________________________________________________________
block1_con

In [ ]:
datagen = ImageDataGenerator(
#     featurewise_center=True,
#     featurewise_std_normalization=True,
#     rotation_range=20,
#     width_shift_range=0.2,
#     height_shift_range=0.2,
#     horizontal_flip=True)
)

datagen.fit(train)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'], )
model.fit_generator(datagen.flow(train, train_label, batch_size=64), epochs=20,  validation_data=(valid, valid_label))

Epoch 1/20
512/512 [==============================] - 217s 425ms/step - loss: 0.9761 - acc: 0.6874 - val_loss: 0.6172 - val_acc: 0.7686
Epoch 2/20
512/512 [==============================] - 211s 411ms/step - loss: 0.4531 - acc: 0.8203 - val_loss: 0.5661 - val_acc: 0.7870
Epoch 3/20
512/512 [==============================] - 210s 410ms/step - loss: 0.3784 - acc: 0.8464 - val_loss: 0.5107 - val_acc: 0.7943
Epoch 4/20
512/512 [==============================] - 209s 409ms/step - loss: 0.3401 - acc: 0.8575 - val_loss: 0.6232 - val_acc: 0.7629
Epoch 5/20
512/512 [==============================] - 209s 409ms/step - loss: 0.3078 - acc: 0.8743 - val_loss: 1.2598 - val_acc: 0.6708
Epoch 6/20
512/512 [==============================] - 209s 408ms/step - loss: 0.2740 - acc: 0.8882 - val_loss: 0.6829 - val_acc: 0.7808
Epoch 7/20
511/512 [============================>.] - ETA: 0s - loss: 0.2359 - acc: 0.9044